In [376]:
import pandas as pd
from collections import Counter, defaultdict
import tensorflow as tf
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [418]:

train = pd.read_csv('/home/sanala/Juputer try/pet/data/amh/train.tsv', sep="\t")
dev = pd.read_csv('/home/sanala/Juputer try/pet/data/amh/dev.tsv', sep="\t")
test = pd.read_csv('/home/sanala/Juputer try/pet/data/amh/test.tsv', sep="\t")

train.info()
train.head(10)
dev.info()     

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1311 entries, 0 to 1310
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  1311 non-null   object
 1   headline  1311 non-null   object
 2   text      1311 non-null   object
 3   url       1311 non-null   object
dtypes: object(4)
memory usage: 41.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188 entries, 0 to 187
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  188 non-null    object
 1   headline  188 non-null    object
 2   text      188 non-null    object
 3   url       188 non-null    object
dtypes: object(4)
memory usage: 6.0+ KB


In [419]:
#There are repeated examples in the training dataset
print('Unique comments: ', train.iloc[:,1].nunique()==train.shape[0] )
print('Null values: ', train.isnull().values.any())
     

Unique comments:  False
Null values:  False


In [420]:
dev['category'].value_counts()

politics    50
health      50
sports      47
business    41
Name: category, dtype: int64

In [380]:
#Usually the headlines are sort but outliers can appear.
print('average sentence length: ', train.iloc[:,1].str.split().str.len().mean())
print('stdev sentence length: ', train.iloc[:,1].str.split().str.len().std())
print('max sentence length: ', train.iloc[:,1].str.split().str.len().max())

average sentence length:  15.776692254012561
stdev sentence length:  3.7644082162769927
max sentence length:  32


In [381]:
train[train.iloc[:,1].duplicated(keep=False).values]

,category,headline,text,url


In [382]:
train_nodup=train.drop_duplicates(subset=['headline'], keep='first')


In [383]:
train_nodup

,category,headline,text,url
0,health,Tí ìgbẹ́ tóò ń yà bá rí báyìí? Àìsàn jẹjẹrẹ ik...,"Ìlera ní Yorùbá pè ní oògùn ọrọ̀, àláfíà ẹni k...",https://www.bbc.com/yoruba/articles/c0w5qxr417zo
1,health,Breastfeeding: Wo àwọn oúnjẹ́ tí yóò mú kí omi...,"Ọpọlọpọ ounjẹ lo wa, to le ṣe awọn iyalọmọ to ...",https://www.bbc.com/yoruba/awon-iroyin-miran-5...
2,religion,Pastor Adeboye: Ìyàwó Adeboye ní ó jẹ́ okùnrin...,"Iyawo pasitọ Enoch Adeboye, Foluke Adeboye ti ...",https://www.bbc.com/yoruba/56262187
3,health,"Charles Olumo Agbako ń ṣe àìsàn ńlá, ó ní ara ...","Òdú ni gbajúgbajà òṣèré tíátà nnì, Abdulsalam ...",https://www.bbc.com/yoruba/articles/cjrn5w951r4o
4,religion,US Fake Marriage Pastor: Ẹ̀wọ̀n ọdún márùn-ún ...,"Olusọagutan kan ni Maryland lorilẹede Amẹrika,...",https://www.bbc.com/yoruba/afrika-59033802
...,...,...,...,...
1428,religion,Pastor Adeboye on 2023 Election: Wọ́n bi mi pé...,"""Mo ni bawo lo ṣe da ọ loju pe Naijiria yoo wa...",https://www.bbc.com/yoruba/afrika-59202960
1429,religion,Silhouette Challenge: Pásítọ̀ Adeboye korò ojú...,Ọpọlọpọ eniyan lo ti n fi ero wọn han lori awo...,https://www.bbc.com/yoruba/media-55868880
1430,sports,Ukraine crisis: FIFA pàṣẹ pé Russia kò gbọdọ̀ ...,FIFA pàṣẹ pé Russia kò gbọdọ̀ díje pẹ̀lú àṣíá ...,https://www.bbc.com/yoruba/60554121
1431,politics,Ayo Fayose: Gómìnà àná ìpínlẹ̀ Ekiti tọrọ àfor...,"A kii rin, ki ori maa ma mi. Eyi lo difa fun ...",https://www.bbc.com/yoruba/afrika-50739133


In [384]:
dev[dev.iloc[:,1].duplicated(keep=False).values]

,category,headline,text,url


In [385]:
dev_nodup=dev.drop_duplicates(subset=['headline'], keep='first')
 

In [414]:
#We take X examples from each label, and the rest become the unlabeled data-set.
classes = train['category'].unique()
samples_per_class=50 #5 10 20 50
# create an empty list to store the randomly selected samples
random_samples = []

# iterate through each class
for class_name in classes:
    # select all rows with the current class
    class_rows = train_nodup[train_nodup['category'] == class_name]
    # randomly select X rows from the class_rows DataFrame
    random_class_samples = class_rows.sample(samples_per_class)
    # add the randomly selected samples to the list
    random_samples.append(random_class_samples)
random_samples_df = pd.concat(random_samples).sample(frac=1)
new_train=random_samples_df.reset_index(drop=True)
unlabeled_train = train.drop(new_train.index)
#new_train
#unlabeled_train

In [415]:
#new_train.columns = [0, 1, 2,3]
#unlabeled_train.columns = [0, 1, 2,3]

In [ ]:
random_samples = []

for class_name in classes:
    class_rows = dev_nodup[dev_nodup['category'] == class_name]
    random_class_samples = class_rows.sample(samples_per_class)
    random_samples.append(random_class_samples)
random_samples_df = pd.concat(random_samples).sample(frac=1)
new_dev=random_samples_df.reset_index(drop=True)
#new_dev.columns = [0, 1, 2,3]

In [417]:
unlabeled_train.insert(0, 'unlabeled', 'unlabeled')
unlabeled_train.reset_index(drop=True)

,unlabeled,category,headline,text,url
0,unlabeled,sports,Diego Maradona: Báyìí ni àgbàọ̀jẹ̀ agbábọ́ọ̀lù...,Ba a ku laa dere eeyan o sunwọn laaye. Agbaọjẹ...,https://www.bbc.com/yoruba/ere-idaraya-55078990
1,unlabeled,politics,Kogi PDP Primaries: Wada fẹ̀yìn Dino gbálẹ̀ ní...,Amoju ẹro Musa Wada la Sẹnẹtọ Dino Melaye mọ l...,https://www.bbc.com/yoruba/afrika-49583650
2,unlabeled,health,Snail Slim Soap: Wo ọ̀pọ̀ àǹfààní tó wà nínú l...,"Ara meeriri ree, a ti ri oju ologbo lori atẹ, ...",https://www.bbc.com/yoruba/afrika-57373685
3,unlabeled,entertainment,"Yollywood: Baba Wande, Kareem Adepoju, Jide Ko...",Iriri kọja ẹgbẹ abẹwu ni ọrọ awọn agba osere k...,https://www.bbc.com/yoruba/62648794
4,unlabeled,entertainment,Dino Melaye vs Iyabo Ojo: Sẹ́nétọ̀ ní lórí irọ...,"Laipẹ yii ni fidio kan gbode lori ayelujara, n...",https://www.bbc.com/yoruba/afrika-57628945
...,...,...,...,...,...
1178,unlabeled,religion,Pastor Adeboye on 2023 Election: Wọ́n bi mi pé...,"""Mo ni bawo lo ṣe da ọ loju pe Naijiria yoo wa...",https://www.bbc.com/yoruba/afrika-59202960
1179,unlabeled,religion,Silhouette Challenge: Pásítọ̀ Adeboye korò ojú...,Ọpọlọpọ eniyan lo ti n fi ero wọn han lori awo...,https://www.bbc.com/yoruba/media-55868880
1180,unlabeled,sports,Ukraine crisis: FIFA pàṣẹ pé Russia kò gbọdọ̀ ...,FIFA pàṣẹ pé Russia kò gbọdọ̀ díje pẹ̀lú àṣíá ...,https://www.bbc.com/yoruba/60554121
1181,unlabeled,politics,Ayo Fayose: Gómìnà àná ìpínlẹ̀ Ekiti tọrọ àfor...,"A kii rin, ki ori maa ma mi. Eyi lo difa fun ...",https://www.bbc.com/yoruba/afrika-50739133


In [410]:
unlabeled_train.drop(columns='category', inplace=True)
unlabeled_train

,unlabeled,headline,text,url
100,unlabeled,Duncan Mighty: Ọlọ́pàá fi pampẹ́ mú olórín tàk...,Ajọ ọlọpaa ipinlẹ Imo ti fi pampẹ ofin mu gbaj...,https://www.bbc.com/yoruba/50795323
101,unlabeled,Arsenal fọ́ Sevilla lẹ́nu pẹ̀lú góòlù 6-0,Goolu mẹta ṣangiliti ni atamatase tuntun ti ẹg...,https://www.bbc.com/yoruba/articles/clmkgxglrl9o
102,unlabeled,TB Joshua burial: Wo àwọn gbajúmọ̀ òṣèré tó lọ...,Eto ayẹyẹ isin idagbere ati isinku gbajugbaja ...,https://www.bbc.com/yoruba/awon-iroyin-miran-5...
103,unlabeled,Aisha Lawal: Ẹni tó kéde pé mo kú yóò rí ìbínú...,Igbe hà hà ni ó gba ẹni àwọn ènìyàn lójú ìpó ì...,https://www.bbc.com/yoruba/media-56926395
104,unlabeled,"Brighton vs Tottenham: Aṣọ́lé Tottenham, Hugo ...","Kaka ki ewe agbọn dẹ, niṣe lo n le koko sii. K...",https://www.bbc.com/yoruba/ere-idaraya-49946254
...,...,...,...,...
1428,unlabeled,Pastor Adeboye on 2023 Election: Wọ́n bi mi pé...,"""Mo ni bawo lo ṣe da ọ loju pe Naijiria yoo wa...",https://www.bbc.com/yoruba/afrika-59202960
1429,unlabeled,Silhouette Challenge: Pásítọ̀ Adeboye korò ojú...,Ọpọlọpọ eniyan lo ti n fi ero wọn han lori awo...,https://www.bbc.com/yoruba/media-55868880
1430,unlabeled,Ukraine crisis: FIFA pàṣẹ pé Russia kò gbọdọ̀ ...,FIFA pàṣẹ pé Russia kò gbọdọ̀ díje pẹ̀lú àṣíá ...,https://www.bbc.com/yoruba/60554121
1431,unlabeled,Ayo Fayose: Gómìnà àná ìpínlẹ̀ Ekiti tọrọ àfor...,"A kii rin, ki ori maa ma mi. Eyi lo difa fun ...",https://www.bbc.com/yoruba/afrika-50739133


In [411]:

#We take X examples from each label, and the rest become the unlabeled data-set.
#Instead, we could also make the new training data-set following the proportion of examples per label.
#PVP pattern-verbalizer pair (PVP)

In [413]:
new_train.to_csv("/home/sanala/Juputer try/pet/data-fsl/yor/20sample/train.tsv", sep="\t", index=False  ,header=False)
unlabeled_train.to_csv("/home/sanala/Juputer try/pet/data-fsl/yor/20sample/unlabeled.tsv", sep="\t", index=False,header=False)
new_dev.to_csv("/home/sanala/Juputer try/pet/data-fsl/yor/20sample/dev.tsv", sep="\t", index=False,header=False)

### Train your own PET



In [224]:
%cd /home/sanala/Juputer try/pet/


/home/sanala/Juputer try/pet


In [ ]:
python3 cli.py \
--method ipet \ 
--pattern_ids 0 1 2 3 4 \
--data_dir '/home/sanala/Juputer try/pet/data-fsl/eng/5sample/' \
--model_type roberta \
--model_name_or_path 'roberta-base' \
--task_name 'topic-classification' \
--output_dir '/home/sanala/Juputer try/pet/results/eng/model-ipet-5-roberta-base/' \
--do_train \
--do_eval \
--sc_repetitions 1

In [ ]:
 !python3 cli.py \
--method pet \
--pattern_ids 0 1 2 3 4 \
--data_dir '/home/sanala/Juputer try/pet/data-fsl/eng/10sample/' \
--model_type xlm-roberta \
--model_name_or_path 'Davlan/afro-xlmr-large' \
--task_name 'topic-classification' \
--output_dir '/home/sanala/Juputer try/pet/results/model-pet-10-Davlan-afro-xlmr-large/' \
--do_train \
--do_eval \
--sc_repetitions 1

Found cached dataset ag_news (/home/sanala/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached processed dataset at /home/sanala/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548/cache-7226dd4188c2c49b.arrow
Loading cached shuffled indices for dataset at /home/sanala/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548/cache-277d3205b905244a.arrow
Loading cached processed dataset at /home/sanala/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548/cache-35dc74a17b998641.arrow
Loading cached shuffled indices for dataset at /home/sanala/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548/cache-4f412e16776cd10f.arrow
Loading cached processed dataset at /home/sanala/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548/cache-693b7ded9f9fd1ab.arrow
Loading cache

[Dataset({
     features: ['text', 'label'],
     num_rows: 8
 }),
 Dataset({
     features: ['text', 'label'],
     num_rows: 8
 }),
 Dataset({
     features: ['text', 'label'],
     num_rows: 8
 }),
 Dataset({
     features: ['text', 'label'],
     num_rows: 8
 })]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

In [144]:
 train_dataset[0]

{'text': 'Stripper shortage reflects industry demands for more demeaning acts: expert (Canadian Press) Canadian Press - OTTAWA (CP) - Contrary to Immigration Department claims, there is no shortage of native-borne exotic dancers in Canada, says a University of Toronto law professor who has studied the strip club business.',
 'label': 0}